In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
import pandas as pd
import html5lib

In [2]:
export_comm_url = "https://commerce-app.gov.in/eidb/ecntcomq.asp"

# Setting up the driver
chrome_options = Options()
chrome_options.headless = True

get_country_list = webdriver.Chrome(options=chrome_options)

get_country_list.get(export_comm_url)

# Populating country list
country_names = []
for o in Select(get_country_list.find_element_by_name('cntcode')).options:
          country_names.append(o.text)
        
get_country_list.close()

In [3]:
export_comm_data = pd.DataFrame()

In [4]:
# Scraping commodity data
def get_export_comm(year, upper_limit=len(country_names)):
    
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(export_comm_url)
    
    global export_comm_data    
    
    for ix in range(0, upper_limit):
        
        country_name = country_names[ix]
        
        Select(driver.find_element_by_id('select2')).select_by_value(str(year)) # Select year

        Select(driver.find_element_by_id('select3')).select_by_visible_text(country_name) # Select country name

        # Additional options    
        driver.find_element_by_id('radioDAll').click()
        driver.find_element_by_id('radiousd').click()

        driver.find_element_by_id('button1').click()

        try:
            tbl = driver.find_element_by_tag_name('table').get_attribute('outerHTML')
            df = pd.read_html(tbl)[0]

            # Scraping dataframe
            df['country'] = country_name
            df['year'] = year
            df.rename(columns={df.columns[4]:'value'}, inplace=True) # Renaming column to use in append


            export_comm_data = export_comm_data.append(df.iloc[:-3,[1,2,4,6,7]], sort=False)
            
        except:
            pass
        
        driver.get(export_comm_url)
    
    driver.close()

In [5]:
get_export_comm(2017)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="select2"]"}
  (Session info: headless chrome=76.0.3809.100)


In [ ]:
export_comm_data.reset_index(drop=True).to_csv('2017_export.csv', index=False)